In [1]:
import pysal as ps
import pandas as pd
import numpy as np
import os
os.chdir("/media/sf_Shared_Linux/Project/")
from dataIO import dbf2df, df2dbf

In [2]:
def SummarizeHAZUS(InputDBF,OutputDBF):
    df = dbf2df(InputDBF)
    df["CensusBloc"]=df["CensusBloc"].astype(str)
    df.set_index("CensusBloc",inplace=True)

    bldgbyblock = {}
    contbyblock = {}
    countbyblock= {}

    for i in range(len(df.index.unique())):
        censusblock = df.index.unique()[i]
        bldgloss = df.ix[censusblock].BldgLoss.sum()
        contloss = df.ix[censusblock].ContLoss.sum()
        bldcount = len(df.BldgLoss[df.index == censusblock])
        bldgbyblock[censusblock] = bldgloss
        contbyblock[censusblock] = contloss
        countbyblock[censusblock]= bldcount

    df_cont = pd.Series(contbyblock, name='ContLoss')
    df_cont.index.name = 'CensusBlock'
    df_bldg = pd.Series(bldgbyblock, name='BldgLoss')
    df_bldg.index.name = 'CensusBlock'
    df_count = pd.Series(countbyblock, name='BldCount')
    df_count.index.name = 'CensusBlock'

    losses = pd.concat([df_bldg,df_cont,df_count],axis=1)
    losses.reset_index(inplace=True)
    losses['CensusBlock'] = losses['CensusBlock'].astype(str)

    my_specs = [('C', 17, 17), ('N', 9, 3), ('N', 9, 3),('N', 3, 0)]
    df2dbf(losses,OutputDBF,my_specs=my_specs)

In [3]:
indbf = ["Data/S0_100yr.dbf","Data/S0_10yr.dbf","Data/S1_10yr.dbf","Data/S1_100yr.dbf"]
outdbf =["Data/S0_100yr_sum.dbf","Data/S0_10yr_sum.dbf","Data/S1_10yr_sum.dbf","Data/S1_100yr_sum.dbf"]
for i in range(4):
    SummarizeHAZUS(indbf[i],outdbf[i])